# Stacks and Queues and Bags

Collections of objects: we want to add, remove, iterate, test if empty.

When we remove, which one? The classics are stacks (LIFO - push and pop), and queue (FIFO - enqueue dequeue).

Subtext: the discipline of modular programming, separating implementation and interface. Enhances reusablity.

## Stacks

Warmup API: stack of strings: `push()` `pop()` `is_empty()` `size()`

Warmup client: Reverse sequence of strings from standard input

In [47]:
# API

class StackOfStrings:
    def __init__(self):
        pass
    
    def push(self, a):
        pass
    
    def pop(self):
        pass


# Client code
# If string = "-", pop and print, else push it onto the stack

stack = StackOfStrings()

a = [1, 2, 5, "-", 3, 4, "-", "-", "-", "-"]

for word in a:
    if word == "-":
        print(stack.pop())
    else:
        stack.push(word)

None
None
None
None
None


### Linked list implementation of stack

Linked list is a series of nodes, where each node contains the item, plus a link to the next Node. You hold a reference to the first Node.

Push creates a new node with the item passed, with it's pointer to the 'old' first node. The first node of the LL now becomes is set to the first node.

Pop sets the first node to the 2nd node, and returns the item from the old first node.

In [43]:
class Node:
    def __init__(self, a):
        self.item = a
        self.next = None

class LinkedList:
    def __init__(self):
        self.first = None
        
    def push(self, item):
        new_node = Node(item)
        new_node.next = self.first
        self.first = new_node
        
    def pop(self):
        item = self.first.item
        self.first = self.first.next
        return item
    
    def is_empty(self):
        return self.first is None

In [45]:
l = LinkedList()
l.push("hello")
l.push("world")
print(l.pop())
print(l.pop())
l.push("restart")
l.push("bleh")
print(l.is_empty())
print(l.pop())
print(l.pop())
l.is_empty()

world
hello
False
bleh
restart


True

In [46]:
# API

class StackOfStrings:
    def __init__(self):
        self.stack = LinkedList()
    
    def push(self, a):
        self.stack.push(a)
    
    def pop(self):
        return self.stack.pop()


# Client code
# If string = "-", pop and print, else push it onto the stack

stack = StackOfStrings()

a = [1, 2, 5, "-", 3, 4, "-", "-", "-", "-"]

for word in a:
    if word == "-":
        print(stack.pop())
    else:
        stack.push(word)

5
4
3
2
1


### Linked List analysis

Proposition: every operation takes constant time in the worst case.

Self-evident: no loops

### Array implementation of stack

This choice between LL and array is fundemental, comes up again and again

Use array s[] to store N items. Push adds a new item at index N, pop removes item from index N-1

(Note intial disadvantage here: overflow underflow possibility - stay tuned)

In [68]:
class ArrayStack:
    def __init__(self, capacity):
        self.array = [None for i in range(capacity)]
        self.N = 0
        
    def push(self, item):
        self.array[self.N] = item
        self.N += 1
        
    def pop(self):
        self.N -= 1
        return self.array[self.N]

In [69]:
# API

class StackOfStrings:
    def __init__(self):
        self.stack = ArrayStack(10)
    
    def push(self, a):
        self.stack.push(a)
    
    def pop(self):
        return self.stack.pop()


# Client code
# If string = "-", pop and print, else push it onto the stack

stack = StackOfStrings()

a = [1, 2, 5, "-", 3, 4, "-", "-", "-", "-"]

for word in a:
    if word == "-":
        print(stack.pop())
    else:
        stack.push(word)

5
4
3
2
1


### Resizing Stack Implementation

Few problems with our stack implementation:
* We should throw an exception on underflow
* Client must provide a capacity - breaks API!
* Loitering: the items stick around in memory after they've been popped - inefficient!

Resizing every time you push: too expensive! Inserting N items takes ~N^2/2 time, because you need to copy all items to a new array every time you increase the array size:

    1 + 2 + 3 + ... + N ~ N^2/2 
    
We want it do be infrequent. We use repeated doubling: if we hit the limit, double the array size and copy stuff into it. Time will be proportional to N on average, not N^2.

    N + (2 + 4 + 8 + ... + N) ~ 3N
    
Every push will be one array access, except a resize operation which will be N.

What about popping: maybe half when N gets to be half the size of C? Bad idea: think about when N = 5 and C = 8. When N goes to 4, you'll halve the array, and if it goes back to 5 you'll just have to double it again! Very inefficient. This is called thrashing.

Instead halve the array when it gets 1/4 full.

In [125]:
class ResizingArrayStack:
    def __init__(self):
        self.array = [None]
        self.N = 0
        self.capacity = 1

        
    def push(self, item):
        if self.N+1 > self.capacity:
            self.array = self.array + [None for i in range(self.capacity)]
            self.capacity = self.capacity * 2
            
        self.array[self.N] = item
        self.N += 1
        #print(f'N: {self.N} C: {self.capacity}')
        print(self.array)
        
    def pop(self):
        self.N -= 1
        item = self.array[self.N]
        self.array[self.N] = None # loitering fix
        
        if self.N <= self.capacity/4:
            self.capacity = int(self.capacity/2)
            self.array = self.array[0:self.capacity]
        #print(f'N: {self.N} C: {self.capacity}')
        print(self.array)

        return item

In [126]:
# API

class StackOfStrings:
    def __init__(self):
        self.stack = ResizingArrayStack()
    
    def push(self, a):
        self.stack.push(a)
    
    def pop(self):
        return self.stack.pop()


# Client code
# If string = "-", pop and print, else push it onto the stack

stack = StackOfStrings()

a = ["to", "be", "or", "not", "to", "-", "be", "-", "-", "that",  "-", "-",  "-", "is"]

for word in a:
    if word == "-":
        print(stack.pop())
    else:
        stack.push(word)

['to']
['to', 'be']
['to', 'be', 'or', None]
['to', 'be', 'or', 'not']
['to', 'be', 'or', 'not', 'to', None, None, None]
['to', 'be', 'or', 'not', None, None, None, None]
to
['to', 'be', 'or', 'not', 'be', None, None, None]
['to', 'be', 'or', 'not', None, None, None, None]
be
['to', 'be', 'or', None, None, None, None, None]
not
['to', 'be', 'or', 'that', None, None, None, None]
['to', 'be', 'or', None, None, None, None, None]
that
['to', 'be', None, None]
or
['to', None]
be
['to', 'is']


Amortised analysis says any sequence of M push and pop operations takes time proportional to M

|x| best | worst | amortised |
|-|-|-|-|
|construct|1|1|1|
|push|1|N|1|
|pop|1|N|1|
|size|1|1|1|

### Tradeoffs

LL: Every operation is constant time, but dealing with the links takes time

RA: Every operation is constant amortized time, but less wasted space